In [1]:
from data import Dataset, get_transfer_dataset
from learner import PredictorLearner, TransferLearner
import analysis as a

dataset_nodejs = Dataset('../dataset/nodejs/43/buffer1.csv', 'nodejs')
dataset_poppler = Dataset('../dataset/poppler/12/cuda.csv', 'poppler')
dataset_x264_src = Dataset('../dataset/x264/43/original_videos_Animation_480P_Animation_480P-087e.csv', 'x264')
dataset_x264_target = Dataset('../dataset/x264/44/original_videos_Animation_480P_Animation_480P-087e.csv', 'x264')
dataset_xz = Dataset('../dataset/xz/23/dickens.csv', 'xz')

In [2]:
### RQ1

predictor_errors = []
transfer_errors = []

X_train, X_validate, y_train, y_validate = dataset_x264_target.get_split_dataset()
predictor = PredictorLearner()
optimised_model, _ = predictor.get_optimal_params(X_validate, y_validate)
predictor.fit(X_train, y_train, premade_model=optimised_model)
predictor_errors.append(predictor.get_error(X_train, y_train))

X_train, X_validate, y_train, y_validate = get_transfer_dataset(dataset_x264_src, dataset_x264_target)
transferrer = TransferLearner()
optimised_model, _ = transferrer.get_optimal_params(X_validate, y_validate)
transferrer.fit(X_train, y_train, premade_model=optimised_model)
transfer_errors.append(transferrer.get_error(X_train, y_train))

print(predictor_errors)
print(transfer_errors)

[array([-0.13775559, -0.26506531, -0.27697002, -0.26424939, -0.35101336])]
[array([-0.44327345, -0.5046296 , -0.380163  , -0.47154408, -0.45512598])]


In [3]:
### RQ3

predictor_training_times_no_cv = []
transfer_training_times_no_cv = []
predictor_training_times = []
transfer_training_times = []

X_train, X_validate, y_train, y_validate = dataset_x264_target.get_split_dataset()
predictor = PredictorLearner()
optimised_model, _ = predictor.get_optimal_params(X_validate, y_validate)
predictor.fit(X_train, y_train, premade_model=optimised_model)
predictor_training_times.append(predictor.get_training_time())

X_train, X_validate, y_train, y_validate = get_transfer_dataset(dataset_x264_src, dataset_x264_target)
transferrer = TransferLearner()
optimised_model, _ = transferrer.get_optimal_params(X_validate, y_validate)
transferrer.fit(X_train, y_train, premade_model=optimised_model)
transfer_training_times.append(transferrer.get_training_time())

print(predictor_training_times)
print(transfer_training_times)

[0.21222615242004395]
[0.0]
